In [ ]:
import nltk
import numpy as np
import random
import string


nltk.download("punkt")
with open("movie_lines.txt/movie_lines.txt", "r", encoding="iso-8859-1") as archivo:
    data = archivo.read()

print(data[:5])